In this notebook, we check if for some knots with at most 15 crossings the 0-surgery is isotopic to the 0-surgery of its mirror (if the knot is not amphicheiral).

In [1]:
import snappy
import csv
import time

def all_positive(manifold):
    '''
    Checks if the solution type of a triangulation is positive.
    '''
    return manifold.solution_type() == 'all tetrahedra positively oriented'

def find_positive_triangulations(manifold,number=1,tries=100):
    '''
    Searches for one triangulation with a positive solution type.
    (Or if number is set to a different value also for different such triangulations.)
    '''
    M = manifold.copy()
    pos_triangulations=[]
    for i in range(tries):
        if all_positive(M):
            pos_triangulations.append(M)
            if len(pos_triangulations)==number:
                return pos_triangulations
            break
        M.randomize()
    for d in M.dual_curves(max_segments=500):
        X = M.drill(d)
        X = X.filled_triangulation()
        X.dehn_fill((1,0),-1)
        for i in range(tries):
            if all_positive(X):
                pos_triangulations.append(X)
                if len(pos_triangulations)==number:
                    return pos_triangulations
                break
            X.randomize()

    # In the closed case, here is another trick.
    if all(not c for c in M.cusp_info('is_complete')):
        for i in range(tries):
            # Drills out a random edge
            X = M.__class__(M.filled_triangulation())
            if all_positive(X):
                pos_triangulations.append(X)
                if len(pos_triangulations)==number:
                    return pos_triangulations
            break
            M.randomize()
    return pos_triangulations

### CHECKING FOR ISOMETRIES

def better_is_isometric_to(X,Y,return_isometries=False,index=100,try_hard=False):
    """
    Returns True if X and Y are isometric.
    Returns False if X and Y have different homologies.
     """ 
    if return_isometries==False:
        w='unclear'
        if X.homology()!=Y.homology():
            return False
        for i in (0,index):
            try:
                w=X.is_isometric_to(Y)
            except (RuntimeError,snappy.SnapPeaFatalError):
                pass
            if w==True:
                return w
            X.randomize()
            Y.randomize()
            i=i+1
        if try_hard:
            pos_triang_X=find_positive_triangulations(X,number=1,tries=index)
            pos_triang_Y=find_positive_triangulations(Y,number=1,tries=index)
            for X in pos_triang_X:
                for Y in pos_triang_Y:
                    w=better_is_isometric_to(X,Y,index=100,try_hard=False)
                    if w==True:
                        return w
        return 'unclear'
    if return_isometries==True:
        w=False
        if X.homology()!=Y.homology():
            return []
        for i in (0,index):
            try:
                w=X.is_isometric_to(Y,return_isometries)
            except (RuntimeError,snappy.SnapPeaFatalError):
                pass
            if w!=False:
                return w
            X.randomize()
            Y.randomize()
            i=i+1
        if try_hard:
            pos_triang_X=find_positive_triangulations(X,number=10,tries=index)
            pos_triang_Y=find_positive_triangulations(Y,number=10,tries=index)
            for X in pos_triang_X:
                for Y in pos_triang_Y:
                    w=better_is_isometric_to(X,Y,return_isometries,index=100,try_hard=False)
                    if w!=False:
                        return w
        return []
    
def orientation_preserving_isometric(M,N,index=100,try_hard=False):
    '''
    Searches for an orientation preserving isometry.
    '''
    for c in M.dual_curves(max_segments=500):
        P=M.drill(c)
        for d in N.dual_curves(max_segments=500):
            Q=N.drill(d)
            for isom in better_is_isometric_to(P,Q,return_isometries=True,index=index,try_hard=try_hard):
                if isom.extends_to_link():
                    if isom.cusp_maps()[0].determinant()==1:
                        return True
    return False

### SYMMETRY GROUP

def better_symmetry_group(M,index=100):
    '''
    This function computes the symmetry group of the input manifold. 
    If the second entry is True it is proven to be the symmetry group.
    '''
    pos_triang=find_positive_triangulations(M,tries=index)
    if pos_triang==[]:
        full=False
        randomizeCount=0
        while randomizeCount<index and full==False:
            try:
                S=M.symmetry_group()
                full=S.is_full_group()
                M.randomize()
                randomizeCount=randomizeCount+1
            except (ValueError, RuntimeError, snappy.SnapPeaFatalError):
                M.randomize()
                randomizeCount=randomizeCount+1
        try:
            return (S,full)
        except NameError:
            return ('unclear',False)
    X=pos_triang[0]
    full=False
    randomizeCount=0
    while randomizeCount<index and full==False:
        try:
            S=X.symmetry_group()
            full=S.is_full_group()
            X.randomize()
            randomizeCount=randomizeCount+1
        except (ValueError, RuntimeError, snappy.SnapPeaFatalError):
            X.randomize()
            randomizeCount=randomizeCount+1
    if full==True:
        return (S,full)
    pos_triang=find_positive_triangulations(M,number=index,tries=index)
    if pos_triang==[]:
        try:
            return (S,full)
        except NameError:
            return ('unclear',False)
    for X in pos_triang:
        full=False
        randomizeCount=0
        while randomizeCount<index and full==False:
            try:
                S=X.symmetry_group()
                full=S.is_full_group()
                X.randomize()
                randomizeCount=randomizeCount+1
            except (ValueError, RuntimeError, snappy.SnapPeaFatalError):
                X.randomize()
                randomizeCount=randomizeCount+1
        if full==True:
            return (S,full)
    try:
        return (S,full)
    except NameError:
        return ('unclear',False)

It is known that the signature of a knot is vanishing if the 0-surgery agrees with the 0-surgery of its mirror. So we first sort out all knots with non-vanishing signature.

In [2]:
start_time=time.time()
signature_vanishing=[]

for K in snappy.HTLinkExteriors(knots_vs_links='knots'):
    D=K.link()
    if D.signature()==0:
        name=K.name()
        signature_vanishing.append(name)
        
print('Number of knots with vanishing signature:',len(signature_vanishing))
print('Time taken: %s minutes ' % ((time.time() - start_time)/60)) 

Number of knots with vanishing signature: 80015
Time taken: 9.11405477921168 minutes 


First we check which of these knots are isotopic to its mirror. (We can sort these out as well.) This we can do via SnapPy by looking at its symmetry group.

In [3]:
start_time=time.time()
isotopic_to_mirror=[]

for knot in signature_vanishing:
    K=snappy.Manifold(knot)
    try:
        S=K.symmetry_group()
        if S.is_full_group():
            if S.is_amphicheiral():
                isotopic_to_mirror.append(knot)
    except:
        pass
    
print('Number of knots that are isotopic to its mirror:',len(isotopic_to_mirror))
print('Time taken: %s minutes ' % ((time.time() - start_time)/60))    

Number of knots that are isotopic to its mirror: 353
Time taken: 1.2427114844322205 minutes 


In [4]:
signature_vanishing_prob_non_amphicheiral=[x for x in signature_vanishing if x not in isotopic_to_mirror]
len(signature_vanishing_prob_non_amphicheiral)

79662

We sort the remaining knots into these that have hyperbolic 0-surgery and these that have non-hyperbolic 0-surgeries. The non-hyperbolic cases can be handeled by looking at their regina names. 

In [5]:
non_hyper_0_surg=['K3a1','K4a1','K5a1','K5a2','K6a3','K7a4','K7a6','K7a7','K8a11','K8a18','K8n1','K8n3','K9a27','K9a36','K9a40','K9a41','K9n5','K10a75','K10a117','K10n13','K10n21','K10n29','K11a247','K11a343','K11a362','K11a363','K11a367','K11n139','K11n141','K12a803','K12a1166','K12a1287','K12n121','K12n582','K12n721','K13a4843','K13a4856','K13a4873','K13a4878','K13n469','K13a3143','K13n3521','K13n3523','K13n3594','K13n3596','K13n3663','K13a4573','K13n4587','K13n4639','K14a12741','K14a17730','K14a19429','K14n3611','K14n18212','K14n19265','K14n21881','K14n21882','K14n22073','K14n22180','K14n22185','K14n22589','K14n24553','K14n26039','K15a54894','K15a78880','K15a84844','K15a84969','K15a85213','K15a85234','K15a85257','K15a85263','K15n19499','K15n40211','K15n41185','K15n43522','K15n48968','K15n51748','K15n59184','K15n72303','K15n112477','K15n112479','K15n113773','K15n113775','K15n113923','K15n115375','K15n115646','K15n124802','K15n142188','K15n153789','K15n156076','K15n160926','K15n164338']

len(non_hyper_0_surg)

92

In [6]:
vanishing_signature_non_hyp=[]
vanishing_signature_hyp=[]

for knot in signature_vanishing_prob_non_amphicheiral:
    if knot in non_hyper_0_surg:
        vanishing_signature_non_hyp.append(knot)
    else:
        vanishing_signature_hyp.append(knot)
        
print('Number of knots with non hyperbolic 0-surgery:',len(vanishing_signature_non_hyp))
print('Number of knots with hyperbolic 0-surgery:',len(vanishing_signature_hyp))

Number of knots with non hyperbolic 0-surgery: 44
Number of knots with hyperbolic 0-surgery: 79618


We first look at the cases where the 0-surgeries are non-hyperbolic. For that we can look at their regina names.

In [7]:
regina_names=[]

with open('regina_names.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        regina_names.append(row)

len(regina_names)

92

In [8]:
for knot in vanishing_signature_non_hyp:
    for x in regina_names:
        if x[0]==knot:
            print(x)

['K6a3', 'SFS [A: (2,1)] / [ 0,1 | 1,-2 ]']
['K8a11', 'SFS [A: (3,1)] / [ 0,1 | 1,-2 ]']
['K8n1', 'SFS [D: (2,1) (2,1)] U/m SFS [D: (3,1) (3,2)], m = [ 0,1 | 1,0 ]']
['K9n5', 'SFS [A: (2,1)] / [ -1,3 | 1,-2 ]']
['K10a75', 'SFS [A: (4,1)] / [ 0,1 | 1,-2 ]']
['K10a117', "JSJ[('SFSpace', 'SFS [A: (2,1)]'), ('SFSpace', 'SFS [A: (3,5)]')]"]
['K10n13', 'SFS [S2: (2,1) (5,1) (10,-7)]']
['K10n29', 'SFS [D: (2,1) (2,1)] U/m SFS [D: (3,1) (3,2)], m = [ -1,2 | 0,1 ]']
['K11n139', 'SFS [A: (2,1)] / [ 2,5 | 1,2 ]']
['K11n141', "JSJ[('hyperbolic', 'm125')]"]
['K12a803', 'SFS [A: (5,1)] / [ 0,1 | 1,-2 ]']
['K12a1166', "JSJ([('SFSpace', 'SFS [A: (2,1)]'), ('SFSpace', 'SFS [A: (4,1)]')])"]
['K12n121', 'SFS [D: (2,1) (2,1)] U/m SFS [D: (2,1) (3,1)], m = [ 5,1 | 4,1 ]']
['K12n582', 'SFS [D: (2,1) (2,1)] U/m SFS [D: (3,1) (3,2)], m = [ -2,3 | -1,2 ]']
['K12n721', "JSJ[('SFSpace', 'SFS [D: (2,1) (2,-1)]'), ('hyperbolic', 'm043')]"]
['K13n469', "JSJ[('SFSpace', 'SFS [D: (2,1) (2,-1)]'), ('hyperbolic', 'm004

If there exists an orientation-reversing diffeomorphism it is isotopic to one that preserves the JSJ decomposition. And in particular there exist orientation reversing-diffeomorphisms of all JSJ pieces. If the piece is a SFS then we can use the classification of SFS with orientation-reversing diffeomorhphisms Section 8 in (https://www.maths.ed.ac.uk/~v1ranick/papers/neumraym.pdf). Thus we can read-off from the regina names directly if a SFS has an orientation-reversing diffeomorphism. By sorting out the SFS that have no orientation-reversing diffeomorphism we are left with the following spaces which all have at least one hyperbolic piece in their JSJ decomposition. We can use SnapPy to check which of these pieces admits an orientation-reversing diffeomorphism.

['K11n141', "JSJ[('hyperbolic', 'm125')]"]

['K12n721', "JSJ[('SFSpace', 'SFS [D: (2,1) (2,-1)]'), ('hyperbolic', 'm043')]"]

['K13n469', "JSJ[('SFSpace', 'SFS [D: (2,1) (2,-1)]'), ('hyperbolic', 'm004')]"]

['K13n3521', "JSJ[('hyperbolic', 'm329')]"]

['K13n3594', "JSJ[('hyperbolic', 'm292')]"]

['K14n3611', "JSJ[('SFSpace', 'M/n2 x~ S1'), ('hyperbolic', 'm015')]"]

['K14n19265', "JSJ[('hyperbolic', 'v3319')]"]

['K14n22185', "JSJ[('SFSpace', 'SFS [D: (2,1) (2,-1)]'), ('hyperbolic', 'm137')]"]

['K14n22589', "JSJ[('hyperbolic', 'm129')]"]

['K14n24553', "JSJ[('SFSpace', 'SFS [D: (2,1) (2,-1)]'), ('hyperbolic', 's663')]"]

['K15n19499', "JSJ[('SFSpace', 'SFS [D: (2,1) (2,-1)]'), ('hyperbolic', 'm032')]"]

['K15n48968', "JSJ[('SFSpace', 'M/n2 x~ S1'), ('hyperbolic', 'v2817')]"]

['K15n51748', "JSJ[('SFSpace', 'SFS [D: (2,1) (2,-1)]'), ('hyperbolic', 'm137')]"]

['K15n72303', "JSJ[('SFSpace', 'M/n2 x~ S1'), ('hyperbolic', 's493')]"]

['K15n112477', "JSJ[('hyperbolic', 's503')]"]

['K15n113773', "JSJ[('hyperbolic', 's843')]"]

['K15n113775', "JSJ[('hyperbolic', 'm129')]"]

['K15n113923', "JSJ[('hyperbolic', 's441')]"]

['K15n115375', "JSJ[('hyperbolic', 'm129')]"]

['K15n153789', "JSJ[('SFSpace', 'SFS [D: (2,1) (2,-1)]'), ('hyperbolic', 'm032')]"]

['K15n164338', "JSJ[('hyperbolic', 's906')]"]

In [9]:
hyperbolic_pieces_to_check=[['K11n141', 'm125'], ['K12n721', 'm043'], ['K13n469', 'm004'], ['K13n3521', 'm329'], ['K13n3594', 'm292'], ['K14n3611', 'm015'], ['K14n19265', 'v3319'], ['K14n22185', 'm137'], ['K14n22589', 'm129'], ['K14n24553', 's663'], ['K15n19499', 'm032'], ['K15n48968', 'v2817'], ['K15n51748', 'm137'], ['K15n72303', 's493'], ['K15n112477', 's503'], ['K15n113773', 's843'], ['K15n113775', 'm129'], ['K15n113923', 's441'], ['K15n115375', 'm129'], ['K15n153789', 'm032'], ['K15n164338', 's906']]

In [10]:
knots_whose_0_surgery_might_admit_reversing_diffeo=[]

for [knot,mfd] in hyperbolic_pieces_to_check:
    M=snappy.Manifold(mfd)
    S=M.symmetry_group()
    if S.is_amphicheiral():
        knots_whose_0_surgery_might_admit_reversing_diffeo.append(knot)
print('Knots whose 0-surgery might admit an orientation-reversing diffeo:',knots_whose_0_surgery_might_admit_reversing_diffeo)

Knots whose 0-surgery might admit an orientation-reversing diffeo: ['K13n469']


So the only possible knot is K13n469 whose 0-surgery is exceptional and given by gluing SFS [D: (2,1) (2,-1)] to the figure eight knot complement m004. Both pieces admit an orientation-reversing diffeomorphism. But we still need to check that the whole manifold does so. This can be done via explicit Kirby calculus methods.

Next, we look at the hyperbolic cases. Here we check with SnapPy if any of these 0-surgeries admits a diffeomorphism that reverses the orientation.

In [30]:
vanishing_signature_hyp[752]

'K13a1'

In [31]:
start_time=time.time()
surgery_isotopic_to_mirror=[]
unclear_symmetry_group=[]

for knot in vanishing_signature_hyp[:751]:
    K=snappy.Manifold(knot)
    K.dehn_fill((0,1))
    [S,w]=better_symmetry_group(K,index=100)
    if w==True:
        if S.is_amphicheiral():
            print('We found a knot whose 0-surgery IS isotopic to its mirror:',knot)
            surgery_isotopic_to_mirror.append(knot)
    if w!=True:
        print('We could NOT compute the symmetry group of the 0-surgery of:',knot)
        unclear_symmetry_group.append(knot)
    if vanishing_signature_hyp.index(knot)%100==0:
        print('Number of knots checked:',vanishing_signature_hyp.index(knot))
        print('Time taken so far: %s minutes ' % ((time.time() - start_time)/60))  

print('Number of knots that have 0-surgery isotopic to its mirror:',len(surgery_isotopic_to_mirror))
print('Number of knots where we could not compute the symmetry group of the 0-surgery:',len(unclear_symmetry_group))
print('Time taken: %s minutes ' % ((time.time() - start_time)/60))   

Number of knots checked: 0
Time taken so far: 0.00023618539174397786 minutes 
Number of knots checked: 100
Time taken so far: 0.3899544358253479 minutes 
Number of knots checked: 200
Time taken so far: 0.6178849498430888 minutes 
Number of knots checked: 300
Time taken so far: 2.8077648878097534 minutes 
Number of knots checked: 400
Time taken so far: 17.08988211552302 minutes 
Number of knots checked: 500
Time taken so far: 118.39791238307953 minutes 
Number of knots checked: 600
Time taken so far: 118.58289406696956 minutes 
Number of knots checked: 700
Time taken so far: 118.65750206708908 minutes 
Number of knots that have 0-surgery isotopic to its mirror: 0
Number of knots where we could not compute the symmetry group of the 0-surgery: 0
Time taken: 118.73113818566004 minutes 
